[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jmdelvecchio/ears33/blob/main/colab_data_tutorials/Probing_Adirondack_landslides_in_Google_Earth_Engine.ipynb)


Google Earth Engine is a powerful tool for Earth observation. There are nearly infinite possibilities for data analysis as the Google servers store petabytes of data and can perform many operations at lightning speed. However, the main vehicle for GEE (and most StackExchange resources, lol) is its JavaScript code editor, which is fine excxept I like Python and many folks like Python. So blessed people like [Qiushang Wu](https://github.com/giswqs) at the University of Tennessee have written some Python packages to make GEE play nicely with our familiar Python syntax and plotting tools. 

Ensure you have registered for a Google Earth Engine account and have some landslide shapfiles in hand to upload to the "Files" second on the left of your screen (the folder icon). 

# Background: NDVI and other spectral indices

Remember from lecture that different land cover exhibits different intensities of visible and invisible wavelengths of energy. Since certain land cover has characteristic high or low reflectances for cetain wavelengths (like green, red and infrared), folks have come up with spectral indices that discriminate between surface cover. One of the most used spectral indices is NDVI, or normalized difference vegetation index. Read about NDVI [here](https://gisgeography.com/ndvi-normalized-difference-vegetation-index/). A huge list of other indices, if you're curious, are [here](https://www.indexdatabase.de/db/i.php). 

# Install and import packages (see note)

There is a supremely dumb thing in Google Colab where a particular package called "ipykernel" is as of April 2022 2.5 years out of date and therefore not compatible with packages getting frequent updates like `geemap` and `eemont`. So you have to do the following:


1.   Run the `pip install` code block
2.   Go to Runtime --> Restart Runtime and hit "ok". This would clear any variables you would have saved but you haven't run anything else yet, so it's fine. 
3. Run the `import` code block. 

Potentially, your runtime might crash and restart (but I didn't experience this while building this notebook). If this is the case **you will need to rerun everything after the `pip install` block.** But let Joanmarie knows if this happens in class. 



In [ ]:
!pip install geopandas --quiet
!pip install shapely --quiet
!pip install geemap --quiet


     |████████████████████████████████| 1.0 MB 13.7 MB/s 
     |████████████████████████████████| 6.3 MB 41.2 MB/s 
     |████████████████████████████████| 16.7 MB 1.9 MB/s 
     |████████████████████████████████| 2.0 MB 15.1 MB/s 
     |████████████████████████████████| 98 kB 6.2 MB/s 
     |████████████████████████████████| 187 kB 52.5 MB/s 
     |████████████████████████████████| 3.0 MB 48.0 MB/s 
     |████████████████████████████████| 99 kB 8.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.0 MB/s 
     |████████████████████████████████| 1.3 MB 57.7 MB/s 
     |████████████████████████████████| 8.7 MB 40.5 MB/s 
     |████████████████████████████████| 130 kB 56.2 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 95 kB 3.8 MB/s 
     |████████████████████████████████| 128 kB 15.8 MB/s 
     |████████████████████████████████| 428 kB 55.2 MB/s 
     |████████████████

Now RESTART RUNTIME! (Ctrl + M + .)

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import numpy as np
from shapely.geometry import Point
from matplotlib import pyplot as plt

import ee
from google.colab import files

import geemap #bless these people

import json

import os

In [ ]:
from google.colab import auth
auth.authenticate_user()

import google
SCOPES = ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/earthengine']
CREDENTIALS, project_id = google.auth.default(default_scopes=SCOPES)

import ee
ee.Initialize(CREDENTIALS, project='my-project')

# Authenticate and initialize GEE

If you haven't registered for GEE, this step will not work. 

Pay close attention to the messages that get displayed when you click through them. Google has made it more secure to get your authentication key but it means you'll get some scary looking messages. Just click "continue" or "yes" whenever it asks *"are you sure??"*


In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=tsjeBos5CoO81X3gtaYtPldHAyovetmfp_7-uRcS0xc&tc=IH8rMmM09uZVcS6jPjapat0hkgUoch0h7IUvIr6mijE&cc=0CnzqCQ5kPe-ARNLiaLvPuN4ibdEPSiMBAcWYmRCg7A

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjMzKQ49jKW5jubfQQmzGO0sSq1e_-8_VDgdEGmLQE2kJ_IPo-aZcI

Successfully saved authorization token.


# Define functions

You've seen functions before, but GEE is big on functions. GEE syntax is set up to leverage the `.map` function, which [maps an algorithm across an ImageCollection](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-map), or collection of satellite scenes that fit some criteria. I think their way is technically pretty Pythonic but it's counter-intuitive when you first think about it. 

The first few are straightforward - they just take an image from the ImageCollection and do something to every image (it's essentially saying `for i in enumerate(images), do something to images[i]`). 

The first function creates a [normalized spectral index](https://developers.google.com/earth-engine/apidocs/ee-image-normalizeddifference), in this case NDVI

In [ ]:
def NAIP_NDVI(image):
  index= image.normalizedDifference(['N','R']).rename('NDVI');
  return image.addBands(index)

In [ ]:
def clip_to_polygon(image):
  return(image).clip(polygon)

In [ ]:
def createTimeBand(image):
  return image.addBands(image.metadata('system:time_start').divide(3.154e10));

I wrote this ugly beast of a function below us in order to hav a somewhat modular tool to make an annual imge from a year's worth of images in an ImageCollection. It says "OK, how do we [reduce](https://developers.google.com/earth-engine/guides/reducers_image_collection) an entire set of images if we want to look at annual trends? Do we take a mean value of what we're intersted in? The max? And over which months?"

In [ ]:
def annual_images(y):
    range_year = ee.Filter.calendarRange(y, y, 'year')
    range_month = ee.Filter.calendarRange(start_month, end_month, 'month')
    filtered_dataset = (index_collection
                        .filter(range_year)
                        .filter(range_month)
                        .map(createTimeBand)) # Needed for linear regression 
    # Combine the mean and standard deviation reducers.
    if analysis == 'mean':
      reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.stdDev(),
        sharedInputs=True
      )
    elif analysis == 'min' or analysis == 'max':
      reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.minMax(),
        sharedInputs=True
      )
    elif analysis == 'median':
      reducers = ee.Reducer.mean().combine(
        reducer2=ee.Reducer.median(),
        sharedInputs=True
      )

# Use the combined reducer to get the mean and SD of the image.
    stats0 = filtered_dataset.reduce(
      reducer=reducers,
    )

    return stats0.set('year',y)

# adapted from https://gis.stackexchange.com/questions/392834/transform-google-earth-engine-script-to-python-with-landsat-8-temporal-data

# Demo: using pre-2009 landslide polygons

## Upload and load in shapefiles

Go to Files on the left and click Upload and select ALL files associated with your landslide shapefile. Here I'm showing you an example of the pre-2009 landslide shapefile that came with the lab. 

In [ ]:
landslides = geemap.shp_to_ee('pre2009.shp')

I'm just going to define a polygon here to mimic the study area from the lab

In [ ]:
polygon = ee.Geometry.Polygon(
  [[[-73.83, 44.18],
   [-73.79, 44.15],
   [-73.87, 44.10],
   [-73.91, 44.13]]]
);

The `geemap.Map()` function creates an instance of an interactive map. If you want to start fresh every time you make a new map, you can call this function where you define your new map as `Map`. If you don't re-instantiate your map, you'll just update the previous `Map` instance. 

In [ ]:
Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
# The empty brackets can take arguments for min and max values for color display 
Map.addLayer(landslides, {}, 'Landslides')
Map.addLayer(polygon, {}, 'Polygon')
Map

## Load in an ImageCollection

[ImageCollections](https://developers.google.com/earth-engine/guides/ic_creating) are how Google stores its imagery and spectral data. They are "geocubes" in that they are spatial data with a number of bands over a number of collection times. You can [filter](https://developers.google.com/earth-engine/guides/ic_filtering) these ImageCollections by spatial or temporal bounds. 

In [ ]:
NAIP = ee.ImageCollection('USDA/NAIP/DOQQ');

naip_2009 = NAIP.filter(ee.Filter.date('2009-01-01', '2009-12-31'))
naip_2013 = NAIP.filter(ee.Filter.date('2013-01-01', '2013-12-31'))
naip_2019 = NAIP.filter(ee.Filter.date('2019-01-01', '2019-12-31'))

In [ ]:
trueColorVis = {
  min: 0.0,
  max: 255.0,
}

Here I am selecting the red (R), green (G), and blue (B) bands to create a true color image. [Select](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-select) lets you choose a subset of an ImageCollection's data layers (most often bands). Read about `addLayer` [here](https://developers.google.com/earth-engine/apidocs/map-addlayer). 

On the resulting map, you can click the wrench icon to open up the interactive toolbar. Clicking on the "Layers" icon to the left of the wrench will open up a menu where you can use the slider to adjust the opacity of each layer. 

In [ ]:
Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
Map.addLayer(naip_2009.select(['R', 'G', 'B']), trueColorVis, '2009');
Map.addLayer(naip_2013.select(['R', 'G', 'B']), trueColorVis, '2013');
Map.addLayer(naip_2019.select(['R', 'G', 'B']), trueColorVis, '2019');
Map

Map(center=[44.137148, -73.849799], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

## Add a spectral index to the ImageCollection

Here we will employ that `.map` function I mentioned earlier. I am going to filter the NAIP data by date starting in 2013 when they began collection of the infrared band (no infrared and therefore no NDVI before 2013).

In [ ]:
adk_ndvi_all= NAIP.filter(ee.Filter.date('2013-01-01', '2019-12-31')).map(NAIP_NDVI)

Here I am adding the year-specific NAIP collections and then selecting the NDVI band I created. 

In [ ]:
Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
Map.addLayer(naip_2019.map(NAIP_NDVI).select(['NDVI']), {}, '2019 NDVI');
Map.addLayer(naip_2013.map(NAIP_NDVI).select(['NDVI']), {}, '2013 NDVI');
Map

## Reduce spectral index data to an annual image

OK, so this is where you can use my state-of-the-art inefficient code to take any ImageCollection you want, select one of its bands, and reduce each year's worth of imagery to a single image per year. 

This code is a little mismatched with NAIP because NAIP imagery is collected only once over a location every three years. I originally wrote this script to look at MODIS data, which came as an average value every 16 days, so in the Arctic I wanted to know what they yearly maximum NDVI was over my field site per year. 

In [ ]:
# You can adjust these numbers, but I wouldn't recommend it to answer the questions
# in this data tutorial.
# If you get an error or no data, you won't find out here. 

# Options are 'mean', 'median', 'min' 'max'
analysis = 'max'

start_year=2013
end_year=2019
start_month=7
end_month=9
index_collection = adk_ndvi_all.select(['NDVI'])

# Don't make any adjustments below here 

years = ee.List.sequence(start_year,end_year)

yearwise_ndvi = years.map(annual_images)

# Make an ImageCollection from the list of images you just composited,
# since you need an ImageCollection for the linear fit reduction
yearCompCol = ee.ImageCollection.fromImages(yearwise_ndvi)

## Perform a pixelwise regression across the ImageCollection to obtain a trend in the spectral index over the observation period

Read about this function [here](https://developers.google.com/earth-engine/guides/reducers_regression). 

In [ ]:
# Get a pixelwise linear regression across the composited ImageCollection
# "select" is time and the band you are interested in
# The output is the slope of the line fit to each pixel's data over time
# and the timestep is "per year"
trend = yearCompCol.select(['system:time_start_mean', 'NDVI_' + analysis]).reduce(ee.Reducer.linearFit())

# 'system:time_start_mean' is my hacky way of doing time per scene
# The value is "the mean number of years since 1970 across the scene"
# which will just be the middle of the month(s) you chose in the year you chose

# The result is two outputs: "scale" is the slope and "offset" is the intercept

## Visualize results

Here are some ancillary topographic visualization collections for your `Map`!

In [ ]:
elevation = ee.Image("USGS/3DEP/10m").select('elevation');
hillshade = ee.Terrain.hillshade(elevation);
slope = ee.Terrain.slope(elevation);


In [ ]:
import geemap.colormaps as cm
palette = cm.palettes.ndvi

Map = geemap.Map()
Map.setCenter(-73.849799,  44.137148, 12);
Map.addLayer(slope, {'min': 0.0, 'max':60.0, 'palette': ['white', 'black']}, '3DEP Hillshade');
Map.addLayer(trend.select('scale'), {'min': -0.1, 'max':0.1, 'palette': palette}, 'NDVI trend')
Map.addLayer(naip_2013.select(['R', 'G', 'B']), trueColorVis, '2013');
Map.addLayer(naip_2019.select(['R', 'G', 'B']), trueColorVis, '2019');
Map.addLayer(landslides, {}, 'Landslides')
Map

See any new slides since 2013??

## Tabulate results for matplotlib-style plotting

Here is a script will [sample](https://developers.google.com/earth-engine/apidocs/ee-image-sample) the pixels in every landslide polygon, and then makes a histogram. 

In [ ]:
my_sample = trend.sample(landslides, 1)

In [ ]:

import geemap.chart as chart
options = {
    "title": 'Pixelwise Yearly NDVI trend for pre-Irene ADK Landslide Scars, 2013-2019',
    "xlabel": 'NDVI trend (NDVI change/year)',
    "ylabel": 'Pixel count',
    "colors": ['#1d6b99'],
}
chart.feature_histogram(my_sample, 'scale', **options)

I wrote this script for another application (spectral index trends in Arctic watersheds) but I figured I'd show this off in case you were curious. This script make a [histogram](https://developers.google.com/earth-engine/apidocs/ee-reducer-autohistogram) for each polygon (identified by `OBJECTID` or similar field in shapefile attribute table) and uses dictionaries and keys to pull that info out of the `autoHistogram` reducer in GEE. 

In [ ]:
lanslide_histos = trend.reduceRegions(collection=landslides,reducer=ee.Reducer.autoHistogram(maxBuckets=50), scale=1)

In [ ]:
info = lanslide_histos.getInfo()
hist_dict = {features['properties']['OBJECTID']:features['properties']['scale'] for features in info['features']}

In [ ]:
fig, ax = plt.subplots(figsize=(16,6))
for key in hist_dict:
  # First element in dictionary value is the bin edge
  bin_edges = [i[0] for i in hist_dict[key]]
  # Second element in dictionary value is counts in each bin
  bin_counts = [i[1] for i in hist_dict[key]]
  bin_counts_norm = bin_counts/np.max(bin_counts)
  ax.plot(bin_edges, bin_counts_norm, alpha=0.5)

#ax.set_xlim((-0.025, 0.025))
ax.set_xlabel("Linear trend of NDVI, " + str(start_year)+"-"+str(end_year))
ax.set_ylabel("Normalized frequency")
fig.suptitle("Distribution of Pixelwise Trends in NDVI for Each Landslide Scar")

# Your turn: post-Irene landslide polygons

Now I want you to see the spectral signatures and trends of the landslides you mapped in lab.

What are the codeblocks you need to run in this section to reproduce those results?


1.   Upload your post-Irene shapefiles and load them in to Google Earth Engine format with `geemap.shp_to_ee`
2.   Use those shapefiles to extract new data
3. Make a new histogram (or other data visualization method of your choice) to see annual trends in NDVI for your newer landslides

You do NOT have to re-run the annual reducer and trend steps - those won't change jsut because you're asking questions about different areas of the output!



In [ ]:
# code blocks here

# Reflection questions



1.   Why might we use NDVI to track landslide activity? What benefits does this spectral index have over the RGB imagery you used in the lab? What are drawbacks? 
2.   Functionally/in real life, what do the trends in NDVI of the landslide scars tell us about how the landscape is changing? Is there a difference in the NDVI trends between the pre- and post-Irene landslides? What do you think sets that pattern?
3. NAIP imagery has a resolution of 1 m and is collected once every three years for an area of the US. [MODIS](https://modis.gsfc.nasa.gov/data/dataprod/mod13.php) is an instrument aboard satellites that collects multispectral data at 250 m every 1-2 days. What are the benefits and drawbacks of using either tool to detect landslides like we did in this activity? (BONUS: for extra points, load in [the MODIS dataset](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1), selecting the built-in NDVI band, and perform this analysis with this different data. You can steal my code from my WISP student researchers [here](https://colab.research.google.com/drive/1vHNcgubyn0HBOBNuFyNPbedKD6Vww9bE).)


Your text here